In [1]:
import Acqu as aq

# Acqu Module
The Acqu module parses in Mk1 and Mk2 data files which can be read event by event. A numpy array is made of form, (ADC,Value) for each event stored as **adcArray**. If the event contains scaler or epics data this can also be read but hasn't been tested recently.


In [2]:
inFile = '/scratch/2019-05_Timepix/Timepix_33.dat'
aq.openFile(inFile)

Mk2 Data


In [3]:
import AcquDetector as aqdet

# AcquDetector module
The detector module, holds all of the information on the detectors and how to convert the raw adc values into relevent detector signals.

Detector information has been converted into the JSON file format. Currently with one file for each measurement parameter (e.g. NaI Energy, NaI Time, BaF Energy, BaF SG Energy, BaF Time, Tagger Time)

The JSON format:
    
    detector   : Unique ID String (e.g. NaI or CB)
    calEq      : python equation which applies calibration to adcs
    channels   : array of channels
        channel    : channel number
        adc        : adc number relating to measurement
        raw        : empty python array raw adc value will be entered into
        value      : empty python array calibrated value will be entered into
        ref        : array number of reference channel if required by calEq
        neighbours : necessary if being interogated as a graph
        [consts]   : any values required by calibration or analysis
    references : array of required reference adcs
        adc        : adc number relating to reference channel
        raw        : empty python array raw adc value will be entered into
       
Acqu setup files can be easily converted using python scripts.

The calEq which allows the detector to self calibrate is possibly not very user friendly.

An example of the first channel of the CB for Mk1 Data:

{
  "detector": "CB", 
  "parameter": "Energy", 
  "calEq": "return [(float(y['raw'][1]-y['raw'][0]))*y['scale']+y['offset'] for y in x]"
  "channels": [
    {
      "scale": 0.0605896, 
      "value": [], 
      "raw": [], 
      "neighbours": [
        1, 
        144, 
        576, 
        145, 
        577, 
        432, 
        579, 
        2, 
        3, 
        146, 
        288
      ], 
      "adc": 3015, 
      "offset": 0.2, 
      "position": [
        3.619, 
        45.214, 
        2.629
      ], 
      "channel": 0
    }
}

Detector files are loaded in a list or file with the file locations:

In [4]:
aqdet.LoadDetectors(['taggerNew.json'])

taggerNew.json


# Calibrating the adc values

Calling Calibrate on the raw adc array filters the array into the different detectors loaded and applies the calibration equation returning a python dictionary containing a numpy array with all of each channels parameters.

In [5]:
import ROOT
import numpy as np
from rootpy.plotting import histogram, Hist2D, Hist, Canvas

taggerChannels = aqdet.Channels['Tagger']
taggerchan     = Hist(taggerChannels,0,taggerChannels,name='taggerChan')
taggerenergy   = Hist(taggerChannels*2,100,1460,name='taggerEne')
taggerchantime = Hist2D(1000,400,1430,taggerChannels,0,taggerChannels,name='taggerChanTime')

def plotCalTagger():
    data = aqdet.Calibrate(aq.adcArray)    
    taggerTimes = aqdet.Arrays['Tagger']['Time']
    
    taggerchan.fill_array(taggerTimes['channel'])
    taggerenergy.fill_array(taggerTimes['Energy'])
    for dat in taggerTimes:
        for time in dat['Time']:
            taggerchantime.Fill(time,dat['channel'])
        
    if(aq.eventNo%1000==0):
        print("Events Processed: ",aq.eventNo)
    
aq.runFunction(plotCalTagger,0,20000)

ROOT.enableJSVis()
c1 = ROOT.TCanvas("my1","The Canvas Title",800,600)
taggerchan.Draw("colz")
taggerchan.GetXaxis().SetTitle("tagger channel")
c1.Draw()
c2 = ROOT.TCanvas("my2","The Canvas Title",800,600)
taggerenergy.Draw("colz")
taggerenergy.GetXaxis().SetTitle("tagger energy")
c2.Draw()
c3 = ROOT.TCanvas("my3","The Canvas Title",800,600)
taggerchantime.Draw("colz")
taggerchantime.GetXaxis().SetTitle("tagger time")
taggerchantime.GetYaxis().SetTitle("tagger channel")
c3.Draw()

Welcome to JupyROOT 6.16/00
('Events Processed: ', 1000)
('Events Processed: ', 2000)
('Events Processed: ', 3000)
('Events Processed: ', 4000)
('Events Processed: ', 5000)
('Events Processed: ', 6000)
('Events Processed: ', 7000)
('Events Processed: ', 8000)
('Events Processed: ', 9000)
('Events Processed: ', 10000)
('Events Processed: ', 11000)
('Events Processed: ', 12000)
('Events Processed: ', 13000)
('Events Processed: ', 14000)
('Events Processed: ', 15000)
('Events Processed: ', 16000)
('Events Processed: ', 17000)
('Events Processed: ', 18000)
('Events Processed: ', 19000)
('Events Processed: ', 20000)
